In [114]:
import os
from dotenv import load_dotenv

load_dotenv()

OPEN_AI_KEY = os.getenv("OPENAI_API_KEY")

MODEL = "gpt-4o-mini"
# MODEL = "llama3.1"

In [115]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPEN_AI_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model="llama3.1")

model.invoke("Tell me a joke")

AIMessage(content='Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e8aaeb6-2c02-4a35-a073-3f372b8de3a9-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29})

In [116]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

"Why don't skeletons fight each other? \n\nThey don't have the guts!"

In [117]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ece380_notes.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'ece380_notes.pdf', 'page': 0}, page_content='ECE 380: Control Systems\nCourse Notes: Winter 2014\nProf. Shreyas Sundaram\nDepartment of Electrical and Computer Engineering\nUniversity of Waterloo'),
 Document(metadata={'source': 'ece380_notes.pdf', 'page': 1}, page_content='ii\nc⃝Shreyas Sundaram'),
 Document(metadata={'source': 'ece380_notes.pdf', 'page': 2}, page_content='Acknowledgments\nParts of these course notes are loosely based on lecture notes by Professors\nDaniel Liberzon, Sean Meyn, and Mark Spong (University of Illinois), on notes\nby Professors Daniel Davison and Daniel Miller (University of Waterloo), and\non parts of the textbook Feedback Control of Dynamic Systems (5th edition) by\nFranklin, Powell and Emami-Naeini. I claim credit for all typos and mistakes\nin the notes.\nThe L ATEX template for The Not So Short Introduction to L ATEX 2εby T. Oetiker\net al. was used to typeset portions of these notes.\nShreyas Sundaram\nUniversity of Wa

In [118]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know". 

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
# print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't answer the question, reply "I don't know". 

Context: Here is some context

Question: Here is a question



In [119]:
chain = prompt | model | parser

In [120]:
chain.input_schema.schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [121]:
chain.invoke(
    {
        "context": "2x + 6 = 10",
        "question": "What is x?"
    }
)

'To solve for x in the equation 2x + 6 = 10, you can follow these steps:\n\n1. Subtract 6 from both sides:\n   2x + 6 - 6 = 10 - 6\n   2x = 4\n\n2. Divide both sides by 2:\n   2x / 2 = 4 / 2\n   x = 2\n\nSo, x = 2.'

In [122]:
from langchain_community.vectorstores import DocArrayInMemorySearch

# would use pinecone instead here
vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [123]:
retriever = vectorstore.as_retriever()

In [124]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    } 
    | prompt 
    | model 
    | parser 
)

In [125]:
questions = [
    "what do you know?",
    "how many pages do you have access to?"
]

for question in questions:
    print(f"Question: {question}")
    print(chain.invoke({"question": question}))

Question: what do you know?
I don't know.
Question: how many pages do you have access to?
I don't know.
